In [1]:
import os
import sys
import torch
import torchvision
from tqdm import tqdm_notebook as tqdm
from transformers import BertModel, BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I0102 06:33:14.142124 140481637357376 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from fgc_support_retri.ser_extractor import *
from fgc_support_retri.utils import read_fgc, read_hotpot
from fgc_support_retri.eval import evalaluate_f1

In [4]:
fgc_items = read_fgc(config.FGC_DEV)

no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次簽訂的北美貿易協定從簽署至生效過了幾日?', 'SENTS': [{'text': '第二次签订的北美贸易协定从签署至生效过了几日?', 'start': 0, 'end': 23}], 'ANSWER': [{'ATEXT': '資訊不足無法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}], 'ATEXT_CN': '资讯不足无法判定'}], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '第二次签订的北美贸易协定从签署至生效过了几日?'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源為何?', 'SENTS': [{'text': '「雅婷逐字稿」的命名起源为何?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「雅婷逐字稿」的命名起源为何?'}
no gold supporting evidence
{'QID': 'D086Q03', 'QTYPE': '申论', 'QTEXT': '不可再生能源的意義是什麼？', 'SENTS': [{'text': '不可再生能源的意义是什么？', 'start': 0, 'end': 13}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Sing

In [5]:
fgc_items[0]

{'QID': 'D004Q01',
 'SENTS': [{'text': '元祐元年（1086年），', 'start': 0, 'end': 12},
  {'text': '宋哲宗即位，', 'start': 12, 'end': 18},
  {'text': '高太皇太后垂帘听政，', 'start': 18, 'end': 28},
  {'text': '回朝任礼部郎中、中书舍人、翰林学士，', 'start': 28, 'end': 46},
  {'text': '元祐四年（1089年）拜龙图阁学士，', 'start': 46, 'end': 64},
  {'text': '曾出任杭州、颍州等知州职务，', 'start': 64, 'end': 78},
  {'text': '官至礼部尚书。', 'start': 78, 'end': 85},
  {'text': '\n绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。', 'start': 85, 'end': 114},
  {'text': '\n元符三年（1100年），', 'start': 114, 'end': 127},
  {'text': '宋徽宗即位，', 'start': 127, 'end': 133},
  {'text': '向太后垂帘听政，', 'start': 133, 'end': 141},
  {'text': '下诏让苏轼北还。', 'start': 141, 'end': 149},
  {'text': '\n建中靖国元年（1101年），', 'start': 149, 'end': 164},
  {'text': '夏天因冷饮过度，', 'start': 164, 'end': 172},
  {'text': '下痢不止，又误服黄芪，', 'start': 172, 'end': 183},
  {'text': '结果病情恶化，', 'start': 183, 'end': 190},
  {'text': '「齿间出血如蚯蚓者无数」，', 'start': 190, 'end': 203},
  {'text': '七月二十八日于常州孙氏馆病卒，', 'start': 203, 'end': 218},
  {'text': 

In [6]:
extractor = SER_sent_extract_V1()

predictions = []
predictions = extractor.predict(fgc_items)
precision, recall, f1 = evalaluate_f1(fgc_items, predictions)
print(precision)
print(recall)
print(f1)

I0102 06:33:19.303287 140481637357376 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I0102 06:33:19.306402 140481637357376 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

0.056338028169014086
0.024742268041237112
0.034383954154727794


In [8]:
predictions

[[1, 3, 5, 6, 9],
 [3, 5],
 [31],
 [31],
 [19],
 [19],
 [23],
 [31],
 [29],
 [3, 9],
 [16],
 [16],
 [16],
 [16],
 [16],
 [16],
 [16],
 [0, 12],
 [0, 12],
 [12],
 [12],
 [0, 3, 12],
 [0, 3, 12],
 [0, 3, 12],
 [0, 3, 12],
 [3, 12],
 [0, 6],
 [0, 6],
 [1],
 [21, 27],
 [24],
 [28],
 [19, 21],
 [3, 5],
 [14, 24],
 [0, 5, 11],
 [0, 3, 9, 11],
 [0, 21],
 [0, 5, 10],
 [21],
 [0, 11],
 [0, 1, 2],
 [0],
 [0, 5, 9],
 [25],
 [5, 8, 9],
 [12],
 [0, 13],
 [0, 5, 8],
 [5, 9],
 [17],
 [0, 18, 21],
 [9, 12, 20, 22],
 [0, 3, 27],
 [37],
 [0],
 [0, 27],
 [0, 27],
 [0, 13, 20, 29],
 [0, 28],
 [0, 28, 34],
 [28],
 [0, 27],
 [0, 1],
 [6],
 [6, 7, 8, 12, 15],
 [6, 7, 8, 10, 12, 15, 16],
 [6, 7, 8, 12, 15, 16],
 [6, 7, 8, 12, 15, 16, 20],
 [3, 5, 14, 35, 45],
 [50],
 [50],
 [10],
 [50],
 [50],
 [10],
 [35],
 [31, 35],
 [5, 13, 35],
 [44, 50],
 [0],
 [0, 11],
 [0],
 [0],
 [0, 11, 12],
 [0],
 [0, 5],
 [0],
 [0],
 [0],
 [22],
 [0],
 [0],
 [20],
 [20],
 [6],
 [0],
 [15, 19],
 [0, 6, 7, 13],
 [0, 2, 17],
 [6],
 [9

In [7]:
for item in fgc_items:
    
    new_sents = []
    for s_i, s in enumerate(item['SENTS']):
        new_sents.append({s_i: s['text']})
    del item['SENTS']
    item['SENTS'] = new_sents


In [21]:
fgc_items[10]

{'QID': 'D011Q02',
 'SUP_EVIDENCE': [9, 16],
 'QTEXT': '形成於北大西洋的熱帶氣旋，又被稱為什麼？',
 'ANS': '颶風',
 'ASPAN': [{'text': '北大西洋', 'start': 417, 'end': 421},
  {'text': '飓风', 'start': 424, 'end': 426},
  {'text': '热带气旋', 'start': 247, 'end': 251},
  {'text': '台风', 'start': 281, 'end': 283},
  {'text': '台风', 'start': 413, 'end': 415},
  {'text': '北太平洋', 'start': 404, 'end': 408}],
 'sup_prediction': [16],
 'SENTS': [{0: '台风（英语：Typhoon，香港天文台缩写T.；日语：台风/たいふう/taifū；韩语：태풍/台风/taepung）是赤道以北，'},
  {1: '国际换日线以西，'},
  {2: '亚太国家或地区对热带气旋的一个分级。'},
  {3: '在气象学上，'},
  {4: '按世界气象组织定义，'},
  {5: '热带气旋中心持续风速达到12级（即64节或以上、32.7m/s或以上，又或者118km/hr或以上）称为飓风（Hurricane）或其他在地近义字。'},
  {6: '西北太平洋地区采用之近义字乃台风。'},
  {7: '\n广义上，「台风」这个词并非一种热带气旋强度。'},
  {8: '在台湾、日本等地，'},
  {9: '将中心持续风速每秒17.2米或以上的热带气旋（包括世界气象组织定义中的热带风暴、强烈热带风暴和台风）均称台风。'},
  {10: '在非正式场合，'},
  {11: '「台风」甚至直接泛指热带气旋本身。'},
  {12: '当西北太平洋的热带气旋达到热带风暴的强度，'},
  {13: '区域专责气象中心（RSMC）日本气象厅会对其编号及命名，'},
  {14: '名称由世界气象组织台风委员会的14个国家和地区提供。'},
  {15: '\n但在不同海洋上也各自有地区性的名称，'},
  {16: 